In [92]:
import numpy as np
from numpy.linalg import det, eig, pinv, matrix_rank, norm
from mdputils import *

In [43]:
%run mdputils.py

In [4]:
def td_solution(P, R, X, G, L):
    """
    Compute the fixed point of the weight vector for general TD-learning.
    
    P: Transition matrix under a policy
    R: Reward vector under the policy
    X: Feature vector matrix
    G: Scaling matrix for gamma
    L: Scaling matrix for lamba
    """
    assert(is_stochastic(P))
    assert(is_ergodic(P))
    assert(is_diagonal(G))
    assert(is_diagonal(L))
    ns = len(P)
    I = np.eye(ns)
    
    # compute intermediate values: distribution and eligibility trace
    d = stationary(P)
    D = np.diag(d)
    E = pinv(I - mult(P, G, L))
    
    # Solve the system of equations
    b = mult(X.T, D, P_trace, R)
    A = mult(X.T, D, P_trace, (I - np.dot(P, G)), X)
    
    return np.dot(pinv(A), b).astype(np.float)

In [113]:
def random_binary(nrows, ncols, row_sum):
    """Generate a random binary matrix ."""
    tmp = np.zeros(ncols)
    tmp[:row_sum] = 1
    ret = np.zeros((nrows, ncols))
    for i in range(nrows):
        ret[i] = np.random.permutation(tmp)
    return ret

In [119]:
np.sum(random_binary(1000,3,1), axis=0)

array([ 345.,  331.,  324.])

In [ ]:
def feature_matrix(phi, states):
    """Compute the feature matrix for the given states & feature function."""
    return np.array([phi(s) for s in states])

In [ ]:
def scaling_matrix(func, states):
    """Compute the scaling matrix for state-dependent parameters."""
    return np.diag([func(s) for s in states])

In [106]:
P = transition_matrix(3)
G = np.diag((1 - np.eye(3))[2])
L = np.diag(np.zeros(3))

In [107]:
assert(is_ergodic(P))
d = stationary(P)

In [103]:
def inv_potential(P, *mats):
    assert(is_stochastic(P))
    I = np.eye(len(P))
    tmp = np.copy(P)
    for x in mats:
        assert(is_diagonal(x))
        tmp = np.dot(tmp, x)
    return (I - tmp)

def potential(P, *mats, tol=1e-6):
    """Compute the potential matrix, subject to scaling."""
    assert(is_stochastic(P))
    I = np.eye(len(P))
    tmp = np.copy(P)
    for x in mats:
        assert(is_diagonal(x))
        tmp = np.dot(tmp, x)
    ret = np.linalg.pinv(I - tmp)
    ret[np.abs(ret) < tol] = 0 # zero values within tolerance
    return ret

def discount(P, G):
    """The discount matrix (I - P_{\pi} \Gamma)
    NB: non-standard terminology.
    """
    assert(is_stochastic(P))
    assert(is_diagonal(G))
    assert(not(np.all(np.diag(G) == 1)))
    I = np.eye(len(P))
    return I - np.dot(P,G)

def eligibility(P, G, L):
    """The matrix which warps the distribution via traces.
    ret = (I - P_{\pi} \Gamma \Lambda)
    """

def warp(P, G, L):
    """
    The matrix which warps the distribution due to gamma and lambda.
    warp = (I - P_{\pi} \Gamma \Lambda)^{-1}
    NB: "warp matrix" is non-standard terminology.

    P : The transition matrix (under a policy)
    G : Diagonal matrix, diag([gamma(s_1), ...])
    L : Diagonal matrix, diag([lambda(s_1), ...])
    """
    assert(is_stochastic(P))
    assert(is_diagonal(G))
    assert(is_diagonal(L))
    assert(not(np.all(np.diag(G) == 1)))
    I = np.eye(len(P))
    return pinv(I - mult(P, G, L))
    
    
def emphasis(P, G, L, N):
    di = np.dot(N, stationary(P))
    U = potential(P, G)
    V = inv_potential(P, G, L)
    m = np.dot(np.dot(di,U), V)
    print(di)
    print(U)
    print(V)
    print(m)
    return np.diag(m)

In [108]:
emphasis(P, G, L, np.eye(3))

[ 0.36842623  0.27758893  0.35398484]
[[ 1.75576842  0.67639681  0.        ]
 [ 0.42614213  1.59088806  0.        ]
 [ 1.04079665  0.78418311  1.        ]]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
[ 1.13358971  0.96840417  0.35398484]


array([[ 1.13358971,  0.        ,  0.        ],
       [ 0.        ,  0.96840417,  0.        ],
       [ 0.        ,  0.        ,  0.35398484]])

In [109]:
U = potential(P, G)
V = inv_potential(P, G, L)

In [110]:
V

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [111]:
d

array([ 0.36842623,  0.27758893,  0.35398484])

In [112]:
np.dot(d, U)

array([ 1.13358971,  0.96840417,  0.35398484])

In [93]:
eig(R)

(array([ 1.        ,  2.17669365,  1.29775508]),
 array([[ 0.        ,  0.40957357,  0.68107585],
        [ 0.        ,  0.69476143, -0.43832433],
        [ 1.        ,  0.59123265,  0.5865215 ]]))

In [79]:
np.linalg.pinv(np.eye(3) - np.dot(P,G))

array([[  1.53950420e+00,   3.75633924e-01,  -1.94289029e-16],
       [  4.10080079e-01,   1.93494453e+00,  -3.88578059e-16],
       [  6.53083649e-01,   6.16346689e-01,   1.00000000e+00]])

In [56]:
np.all(np.diag(G) == 1)

False

In [62]:
somenone(3)

array([ 1.,  1.,  1.])

In [63]:
def foo(*args):
    print(args)
    print(type(args))

In [64]:
foo()

()
<class 'tuple'>


In [68]:
d[()]

array([ 0.49987624,  0.21488646,  0.28523729])